$sT = 3$

M is a random variable with a lognormal distribution with parameters $\mu$ and $\sigma^2$.

The PDF of M is given by:

$f_M(m) = \frac{1}{m\sigma\sqrt{2\pi}}e^{-\frac{(\ln(m) - \mu)^2}{2\sigma^2}}$


Mean Service Time
$E[t_s] = sT * E[M] = sT * e^{\mu + \frac{\sigma^2}{2}}$

Variance of the Service Time
$Var(t_s) = Var(sT * M)= sT^2 * (e^{\sigma^2} - 1) * e^{2\mu + \sigma^2}$

For computing the parameters $\mu$ and $\sigma^2$ we use:

$\mu = \ln(\frac{E[M]^2}{\sqrt{Var(M) + E[M]^2}})$

$\sigma^2 = \ln(1 + \frac{Var(M)}{E[M]^2})$

In [50]:
import math 
from scipy.stats import lognorm

sT = 3


E_M = 15
VAR_M = 16
K = 13

# Parameters of the lognormal distribution
mu = math.log(E_M**2/math.sqrt(VAR_M+E_M**2))
sigma = math.sqrt(math.log((VAR_M/E_M**2)+1))
print(mu, sigma)

alphaK = lognorm.cdf(K+0.5, s=sigma, scale=math.exp(mu))
print(alphaK)


2.6737019354590927 0.2621002313738675
0.39322033097501774


In [51]:
import pandas as pd
import numpy as np
import math


n = 50
C = 4

VTM = pd.read_csv("VTMLogS3.csv").fillna(0)

VTM = VTM[VTM["type"] == "scalar"]
Exp_S = VTM[VTM["module"].str.contains("SuperMarket.checkout.localSink")]
Exp_S = Exp_S[Exp_S["name"] == "totalServiceTime:mean"]
VTM = VTM[VTM["module"].str.contains("SuperMarket.checkout.queues")]
Exp_NQ = VTM[VTM["name"] == "queueLength:timeavg"]
Exp_RHO = VTM[VTM["name"] == "busy:timeavg"]
Exp_W = VTM[VTM["name"] == "queueingTime:mean"]

CSV = {
    "NQ": Exp_NQ,
    "RHO": Exp_RHO,
    "S": Exp_S,
    "W": Exp_W
}

PI = {}

for key in CSV:
    PI[key] = {}
    for i in range(C):
        temp = []
        temp = [CSV[key]["value"].values[j * C + i] for j in range(n)]
        string = f"Q{i}"
        temp = list(map(float, temp))
        PI[key][string] = temp
    PI[key] = pd.DataFrame(PI[key])


PI_T = {}
for key in PI:
    PI_T[key] = pd.DataFrame()
    PI_T[key]["Mean"] = PI[key].mean()
    # CI 95%
    PI_T[key]["CI (LB)"] = PI[key].mean() - (1.96 * (PI[key].std() / math.sqrt(n)))
    PI_T[key]["CI (UB)"] = PI[key].mean() + (1.96 * (PI[key].std() / math.sqrt(n)))


for key in PI_T:
    print(f"    [{key}]")
    print(PI_T[key])
    print("\n")




    [NQ]
        Mean   CI (LB)   CI (UB)
Q0  0.017224  0.016357  0.018090
Q1  0.017089  0.016079  0.018099
Q2  0.129533  0.124619  0.134447
Q3  0.133819  0.128600  0.139038


    [RHO]
        Mean   CI (LB)   CI (UB)
Q0  0.166524  0.164512  0.168537
Q1  0.166728  0.164293  0.169164
Q2  0.393314  0.388981  0.397647
Q3  0.395599  0.390942  0.400256


    [S]
         Mean    CI (LB)    CI (UB)
Q0  33.938034  33.875137  34.000931
Q1  33.918786  33.864203  33.973369
Q2  52.157149  52.057333  52.256965
Q3  52.192014  52.095066  52.288961


    [W]
         Mean    CI (LB)    CI (UB)
Q0   3.499760   3.345272   3.654248
Q1   3.469159   3.276299   3.662018
Q2  17.146288  16.579999  17.712578
Q3  17.593801  17.027884  18.159719




In [54]:
# C=4 DISCRETE
# This Code uses PI_T from the above cell
from scipy.integrate import quad
import math
import numpy as np

C = 4
p = 0.5
T = 40
t = 1/T
M = E_M
# K = K
# sT = sT
# alphaK = alphaK

def integrand(x):
    return (math.floor(x+0.5)*lognorm.pdf(x, s=sigma, scale=math.exp(mu)))

def integrand_2(x):
    return (((math.floor(x+0.5))**2)*lognorm.pdf(x, s=sigma, scale=math.exp(mu)))

pi_i = alphaK/(C*p)
t_i = t*pi_i

S_i = ((quad(integrand, 0, K+0.5, limit=200)[0])/alphaK)*sT
# print(S_i)
# S_i = E_M * norm.cdf(((math.log(K+0.51) - mu - sigma**2)/sigma)) / norm.cdf(((math.log(K+0.51) - mu)/sigma))
# print(S_i)
s_i = 1/S_i

Var_S_i = ((quad(integrand_2, 0, K+0.5, limit=200)[0])/alphaK)*sT**2 - S_i**2
# print("Var_i", Var_S_i, VAR_M)

rho_i = t_i/s_i
# N_i = rho_i/(1-rho_i)
N_i = rho_i + ((rho_i**2 + t_i**2 * Var_S_i)/(2*(1-rho_i)))
NQ_i = N_i - rho_i
R_i = N_i/t_i
W_i = R_i - S_i


pi_j = (1-alphaK)/(C*(1-p))
t_j = t*pi_j

S_j = ((quad(integrand, K+0.5, np.inf, limit=200)[0])/(1-alphaK))*sT
# print(S_j)
# S_j = E_M * norm.cdf((-(math.log(K+0.50) - mu - sigma**2)/sigma)) / (1-norm.cdf(((math.log(K+0.50) - mu)/sigma)))
# print(S_j)
s_j = 1/S_j

Var_S_j = ((quad(integrand_2, K+0.5, np.inf, limit=200)[0])/(1-alphaK))*sT**2 - S_j**2
# print("Var_j", Var_S_j, VAR_M)

rho_j = t_j/s_j
# N_j = rho_j/(1-rho_j)
N_j = rho_j + ((rho_j**2 + t_j**2 * Var_S_j)/(2*(1-rho_j)))
NQ_j = N_j - rho_j
R_j = N_j/t_j
W_j = R_j - S_j


if "Theoretical" not in PI_T["RHO"].columns:
    PI_T["RHO"].insert(0, "Theoretical", [rho_i, rho_i, rho_j, rho_j])
else:
    PI_T["RHO"]["Theoretical"] = [rho_i, rho_i, rho_j, rho_j]

if "Theoretical" not in PI_T["S"].columns:
    PI_T["S"].insert(0, "Theoretical", [S_i, S_i, S_j, S_j])
else:
    PI_T["S"]["Theoretical"] = [S_i, S_i, S_j, S_j]

if "Theoretical" not in PI_T["NQ"].columns:
    PI_T["NQ"].insert(0, "Theoretical", [NQ_i, NQ_i, NQ_j, NQ_j])
else:
    PI_T["NQ"]["Theoretical"] = [NQ_i, NQ_i, NQ_j, NQ_j]

if "Theoretical" not in PI_T["W"].columns:
    PI_T["W"].insert(0, "Theoretical", [W_i, W_i, W_j, W_j])
else:
    PI_T["W"]["Theoretical"] = [W_i, W_i, W_j, W_j]

for key in PI_T:
    PI_T[key]["OK"] = [""] * 4
    for i in range(4):
        if PI_T[key]["Theoretical"].iloc[i] <= PI_T[key]["CI (UB)"].iloc[i] and PI_T[key]["Theoretical"].iloc[i] >= PI_T[key]["CI (LB)"].iloc[i]:
            PI_T[key].loc[f"Q{i}", "OK"] = "V"
        else:
            PI_T[key].loc[f"Q{i}", "OK"] = ""

print("\n")
for key in PI_T:
    print(f"    [{key}]")
    print(PI_T[key])
    print("\n")


C:\Users\giova\AppData\Local\Temp\ipykernel_1288\64311889.py:45: IntegrationWarning: The maximum number of subdivisions (200) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  S_j = ((quad(integrand, K+0.5, np.inf, limit=200)[0])/(1-alphaK))*sT




    [NQ]
    Theoretical      Mean   CI (LB)   CI (UB) OK
Q0     0.016983  0.017224  0.016357  0.018090  V
Q1     0.016983  0.017089  0.016079  0.018099  V
Q2     0.134086  0.129533  0.124619  0.134447  V
Q3     0.134086  0.133819  0.128600  0.139038  V


    [RHO]
    Theoretical      Mean   CI (LB)   CI (UB) OK
Q0     0.166725  0.166524  0.164512  0.168537  V
Q1     0.166725  0.166728  0.164293  0.169164  V
Q2     0.395776  0.393314  0.388981  0.397647  V
Q3     0.395776  0.395599  0.390942  0.400256  V


    [S]
    Theoretical       Mean    CI (LB)    CI (UB) OK
Q0    33.919894  33.938034  33.875137  34.000931  V
Q1    33.919894  33.918786  33.864203  33.973369  V
Q2    52.180458  52.157149  52.057333  52.256965  V
Q3    52.180458  52.192014  52.095066  52.288961  V


    [W]
    Theoretical       Mean    CI (LB)    CI (UB) OK
Q0     3.455249   3.499760   3.345272   3.654248  V
Q1     3.455249   3.469159   3.276299   3.662018  V
Q2    17.678428  17.146288  16.579999  17.712578  V

C:\Users\giova\AppData\Local\Temp\ipykernel_1288\64311889.py:51: IntegrationWarning: The maximum number of subdivisions (200) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  Var_S_j = ((quad(integrand_2, K+0.5, np.inf, limit=200)[0])/(1-alphaK))*sT**2 - S_j**2
